In [1]:
from glove import GloVe
import torch
import numpy as np
import nltk
from vocabulary import Vocabulary
import pickle
import csv 
import statistics

In [2]:
fields = ['target_one vs target_two', 'attribute_one vs attribute_two', 'd']

In [3]:
embed_200d = GloVe(400000, 200, 100, 0.75)

In [4]:
embed_100d = GloVe(400000, 100, 100, 0.75)

In [6]:
embed_50d = GloVe(400000, 50, 100, 0.75)

In [7]:
embed_200d.load_state_dict(torch.load('/home/rrr/Desktop/glove_code_final/embeddings_200d.pt'))

<All keys matched successfully>

In [8]:
embed_100d.load_state_dict(torch.load('/home/rrr/Desktop/glove_code_final/embeddings_100d.pt'))

<All keys matched successfully>

In [9]:
embed_50d.load_state_dict(torch.load('/home/rrr/Desktop/glove_code_final/embeddings_50d.pt'))

<All keys matched successfully>

In [10]:
with open('/home/rrr/Desktop/glove_code_final/cooccurrence_dir/vocab.pkl', 'rb') as file:
    # Load the object from the file using pickle.load()
    my_object = pickle.load(file)

print(len(my_object))

400000


In [11]:
file = open("weat.txt", "r" ) 
file=file.read()

In [12]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(file.lower())

In [13]:
def cosine_similarity(x, y):
  # if(x is not None):
  #   if(y is not None):
  #     num = np.dot(x, y)
  #     a = np.linalg.norm(x)
  #     b = np.linalg.norm(y)
  #     den = a*b
  #     res = num/den
  #     return res
  return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [23]:
rows = []
mean_list = []

In [14]:
def target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, 
                     attribute_one_words, target_two_words, attribute_two_words, model):
        cos=[]
        s=0
        s1=[]
        s2=[]
        S=[]
        n=0
        for i in range(0, len(target_one_words)):
          c1=[]
          c2=[]
          for k in range(0, len(attribute_one_words)):
            wt = target_one_words[i][:-1]
            at1 = attribute_one_words[k][:-1]
            ind_wt = my_object.get_index(wt)
            ind_at1 = my_object.get_index(at1)
            
            try:
              x = model.weight(torch.tensor(ind_wt))
              y = model.weight(torch.tensor(ind_at1))
              x = x.detach().numpy()
              y = y.detach().numpy()
              cos1= cosine_similarity(x, y)
              cos.append(cos1)
              c1.append(cos1)
            except:
              cos1=0
              cos.append(cos1)
              c1.append(cos1)
              continue
              
          for k in range(0, len(attribute_two_words)):
            cos2=0
            wt = target_one_words[i][:-1]
            at2 = attribute_two_words[k][:-1]
            ind_wt = my_object.get_index(wt)
            ind_at2 = my_object.get_index(at2)
            
            try:
              x = model.weight(torch.tensor(ind_wt))
              y = model.weight(torch.tensor(ind_at2))
              x = x.detach().numpy()
              y = y.detach().numpy()
              cos2= cosine_similarity(x, y)
              cos.append(cos2)
              c2.append(cos2)
            except:
              cos2=0
              cos.append(cos2)
              c2.append(cos2)
              continue
          s1.append((np.mean(c1)-np.mean(c2)))
          S.append((np.mean(c1)-np.mean(c2)))
          n=n+1
            
        for i in range(0, len(target_two_words)):
          c1=[]
          c2=[]

          for k in range(0, len(attribute_one_words)):
            wt = target_two_words[i][:-1]
            at1 = attribute_one_words[k][:-1]
            ind_wt = my_object.get_index(wt)
            ind_at1 = my_object.get_index(at1)
            
            try:
              x = model.weight(torch.tensor(ind_wt))
              y = model.weight(torch.tensor(ind_at1))
              x = x.detach().numpy()
              y = y.detach().numpy()
              cos1= cosine_similarity(x, y)
              cos.append(cos1)
              c1.append(cos1)
            except:
              cos1=0
              cos.append(cos1)
              c1.append(cos1)
              continue
            
          for k in range(0, len(attribute_two_words)):
            cos2=0
            wt = target_two_words[i][:-1]
            at2 = attribute_two_words[k][:-1]
            wt = target_one_words[i][:-1]
            at2 = attribute_two_words[k][:-1]
            ind_wt = my_object.get_index(wt)
            ind_at2 = my_object.get_index(at2)
            
            try:
              x = model.weight(torch.tensor(ind_wt))
              y = model.weight(torch.tensor(ind_at2))
              x = x.detach().numpy()
              y = y.detach().numpy()
              cos2= cosine_similarity(x, y)
              cos.append(cos2)
              c2.append(cos2)
            except:
              cos2=0
              cos.append(cos2)
              c2.append(cos2)
              continue
          s2.append((np.mean(c1)-np.mean(c2)))
          S.append((np.mean(c1)-np.mean(c2)))
        s=np.sum(s1)-np.sum(s2)
        stdev=np.std(S)
        res = s/(stdev*n)
        
        if(target_one == 'european_american_names'):
            res2 = '%.2f' % res
            mean_list.append(res2)
            
            if(len(mean_list) == 3):
                p = []
                val1 = target_one + ' vs ' + target_two
                p.append(val1)
            
                val2 = attribute_one + ' vs ' + attribute_two
                p.append(val2)
                
                float_mean_list = []
                for val in mean_list:
                    float_mean_list.append(float(val))
            
                avg = statistics.mean(float_mean_list)
                res2 = '%.2f' % avg
                p.append(res2)
                rows.append(p)
                
        else:
            p = []

            val1 = target_one + ' vs ' + target_two
            p.append(val1)

            val2 = attribute_one + ' vs ' + attribute_two
            p.append(val2)

            #print(target_one + ' vs ' + target_two  + ' , ' +attribute_one + ' vs ' + attribute_two +', d = ',end="")
            #print('%.2f' % res)
            res2 = '%.2f' % res
            p.append(res2)

            rows.append(p)
        #print(res2)


In [16]:
for i in range(len(raw_sentences)-30):
    words=raw_sentences[i*4].split()
    target_one = words[0][:-1]
    target_one_words = words[1:]
    words1=raw_sentences[i*4+1].split()
    target_two = words1[0][:-1]
    target_two_words = words1[1:]
    words2=raw_sentences[i*4+2].split()
    attribute_one = words2[0][:-1]
    attribute_one_words = words2[1:]
    words3 = raw_sentences[i*4+3].split()
    attribute_two = words3[0][:-1]
    attribute_two_words = words3[1:]
    
    target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, 
                     attribute_one_words, target_two_words, attribute_two_words, embed_200d)

In [ ]:
# name of csv file 
filename = "results_glove_200d.csv"

In [ ]:
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [ ]:
rows = []
mean_list = []

In [20]:
for i in range(len(raw_sentences)-30):
    words=raw_sentences[i*4].split()
    target_one = words[0][:-1]
    target_one_words = words[1:]
    words1=raw_sentences[i*4+1].split()
    target_two = words1[0][:-1]
    target_two_words = words1[1:]
    words2=raw_sentences[i*4+2].split()
    attribute_one = words2[0][:-1]
    attribute_one_words = words2[1:]
    words3 = raw_sentences[i*4+3].split()
    attribute_two = words3[0][:-1]
    attribute_two_words = words3[1:]
    
    target_attribute(target_one,target_two, target_one_words, attribute_one,attribute_two, 
                     attribute_one_words, target_two_words, attribute_two_words, embed_100d)

In [21]:
# name of csv file 
filename = "results_glove_100d.csv"

In [22]:
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)